<a href="https://colab.research.google.com/github/iammook/Recommendation/blob/main/collaborative.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path_res = "/content/drive/MyDrive/Colab Notebooks/Data/rating_final.csv"
path_res_cuisine = "/content/drive/MyDrive/Colab Notebooks/Data/chefmozcuisine.csv"
df_restaurant = pd.read_csv(path_res)
df_restaurant_cuisine = pd.read_csv(path_res_cuisine)
df_restaurant.head(10)

,userID,placeID,rating,food_rating,service_rating
0,1077,135085,2,2,2
1,1077,135038,2,2,1
2,1077,132825,2,2,2
3,1077,135060,1,2,2
4,1068,135104,1,1,2
5,1068,132740,0,0,0
6,1068,132663,1,1,1
7,1068,132732,0,0,0
8,1068,132630,1,1,1
9,1067,132584,2,2,2


In [ ]:
df_restaurant_cuisine.head(10)

,placeID,Rcuisine
0,135110,Spanish
1,135109,Italian
2,135107,Latin_American
3,135106,Mexican
4,135105,Fast_Food
5,135104,Mexican
6,135103,Burgers
7,135103,Dessert-Ice_Cream
8,135103,Fast_Food
9,135103,Hot_Dogs


In [ ]:
#normalized rating for a user
Mean = df_restaurant.groupby(by="userID",as_index=False)['rating'].mean()
Rating_avg = pd.merge(df_restaurant,Mean,on='userID')
Rating_avg['adg_rating']=Rating_avg['rating_x']-Rating_avg['rating_y']
Rating_avg.head(10) 

,userID,placeID,rating_x,food_rating,service_rating,rating_y,adg_rating
0,1077,135085,2,2,2,1.400,0.600
1,1077,135038,2,2,1,1.400,0.600
2,1077,132825,2,2,2,1.400,0.600
3,1077,135060,1,2,2,1.400,-0.400
4,1077,135027,0,1,1,1.400,-1.400
5,1068,135104,1,1,2,0.625,0.375
6,1068,132740,0,0,0,0.625,-0.625
7,1068,132663,1,1,1,0.625,0.375
8,1068,132732,0,0,0,0.625,-0.625
9,1068,132630,1,1,1,0.625,0.375


pre-processing and clean the data

In [ ]:
# Check NaN data
check = pd.pivot_table(Rating_avg,values='rating_x',index='userID',columns='placeID')
check.head(10)

placeID,132560,132561,132564,132572,132583,132584,132594,132608,132609,132613,132626,132630,132654,132660,132663,132665,132667,132668,132706,132715,132717,132723,132732,132733,132740,132754,132755,132766,132767,132768,132773,132825,132830,132834,132845,132846,132847,132851,132854,132856,...,135044,135045,135046,135047,135048,135049,135050,135051,135052,135053,135054,135055,135057,135058,135059,135060,135062,135063,135064,135065,135066,135069,135070,135071,135072,135073,135074,135075,135076,135079,135080,135081,135082,135085,135086,135088,135104,135106,135108,135109
userID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN
1003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
1005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1006,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1007,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN,NaN,1.0,NaN
1008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
1009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,1.0,2.0,1.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
final = pd.pivot_table(Rating_avg,values='adg_rating',index='userID',columns='placeID')
final.head(10)

placeID,132560,132561,132564,132572,132583,132584,132594,132608,132609,132613,132626,132630,132654,132660,132663,132665,132667,132668,132706,132715,132717,132723,132732,132733,132740,132754,132755,132766,132767,132768,132773,132825,132830,132834,132845,132846,132847,132851,132854,132856,...,135044,135045,135046,135047,135048,135049,135050,135051,135052,135053,135054,135055,135057,135058,135059,135060,135062,135063,135064,135065,135066,135069,135070,135071,135072,135073,135074,135075,135076,135079,135080,135081,135082,135085,135086,135088,135104,135106,135108,135109
userID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.888889,-0.111111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-0.111111,NaN,NaN,NaN,NaN,NaN,-0.111111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.111111,NaN,NaN,NaN,NaN,NaN,NaN
1002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.400000,NaN,NaN,NaN,NaN,NaN,-0.400000,NaN,-0.400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.400000,NaN,NaN,NaN,-0.400,NaN,NaN
1003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.384615,NaN,NaN,NaN,0.384615,0.384615,NaN,NaN,NaN,NaN,0.384615,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.384615,NaN,NaN,NaN,-1.615385,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.384615,NaN,0.384615,0.384615,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.875000,0.125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.125,NaN,NaN
1005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-0.333333,NaN,NaN,NaN,NaN,NaN,-0.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1006,NaN,NaN,NaN,-0.363636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.363636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.363636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.363636,NaN,-0.363636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1007,NaN,NaN,NaN,0.111111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.111111,0.111111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.111111,-0.888889,NaN,NaN,NaN,0.111111,NaN
1008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.111111,NaN,-0.111111,0.888889,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.111111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.111111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.111111,NaN
1009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.363636,NaN,NaN,NaN,NaN,NaN,-0.363636,NaN,0.636364,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.636364,NaN,NaN,NaN,NaN,-0.363636,0.636364,-0.363636,NaN,-1.363636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.363636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:

# Replacing NaN by restaurant Average
final_restaurant = final.fillna(final.mean(axis=0))

# Replacing NaN by user Average
final_user = final.apply(lambda row: row.fillna(row.mean()), axis=1)

In [ ]:
final_restaurant.head(10)

placeID,132560,132561,132564,132572,132583,132584,132594,132608,132609,132613,132626,132630,132654,132660,132663,132665,132667,132668,132706,132715,132717,132723,132732,132733,132740,132754,132755,132766,132767,132768,132773,132825,132830,132834,132845,132846,132847,132851,132854,132856,...,135044,135045,135046,135047,135048,135049,135050,135051,135052,135053,135054,135055,135057,135058,135059,135060,135062,135063,135064,135065,135066,135069,135070,135071,135072,135073,135074,135075,135076,135079,135080,135081,135082,135085,135086,135088,135104,135106,135108,135109
userID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1001,-0.090278,0.075,-0.1875,-0.249016,-0.238095,0.225,-0.022222,0.18287,-0.168889,0.217593,0.304167,0.356481,-0.425,0.078333,-0.185185,0.043333,0.025,-0.111111,0.075,-0.291667,0.138889,0.102566,-0.138889,0.243056,-0.28125,0.054509,0.465956,-0.574074,-0.19537,0.244762,0.333333,0.888889,-0.111111,-0.073565,-0.04899,0.224242,-0.006734,0.441198,0.156860,-0.319032,...,-0.203819,-0.111111,0.040391,-0.181433,-0.405132,-0.428449,-0.113068,-0.111111,0.149467,0.111007,0.098965,0.358279,0.268932,-0.097443,0.212522,-0.097037,-0.00283,-0.088900,-0.296135,-0.088113,0.113902,-0.318582,-0.020802,-0.010354,-0.042244,0.203571,0.279514,0.193323,0.093451,0.033527,-0.114033,-0.151945,-0.167312,-1.111111,-0.261950,-0.045833,-0.217857,-0.161554,0.265863,-0.197917
1002,-0.090278,0.075,-0.1875,-0.249016,-0.238095,0.225,-0.022222,0.18287,-0.168889,0.217593,0.304167,0.356481,-0.425,0.078333,-0.185185,0.043333,0.025,-0.111111,0.075,-0.291667,0.138889,0.102566,-0.138889,0.243056,-0.28125,0.054509,0.465956,-0.574074,-0.19537,0.244762,0.333333,0.600000,-0.071065,-0.073565,-0.04899,0.224242,-0.006734,0.441198,0.156860,-0.319032,...,-0.203819,0.195089,0.040391,-0.181433,-0.405132,-0.428449,-0.113068,0.009834,-0.400000,0.111007,0.098965,0.358279,0.268932,-0.097443,-0.400000,-0.097037,-0.40000,-0.088900,-0.296135,-0.088113,0.113902,-0.318582,-0.020802,-0.010354,-0.042244,0.203571,0.279514,0.193323,0.093451,0.033527,-0.114033,-0.151945,-0.167312,-0.400000,-0.261950,-0.045833,-0.217857,-0.400000,0.265863,-0.197917
1003,-0.090278,0.075,-0.1875,-0.249016,-0.238095,0.225,-0.022222,0.18287,-0.168889,0.217593,0.304167,0.356481,-0.425,0.078333,-0.185185,0.043333,0.025,-0.111111,0.075,-0.291667,0.138889,0.384615,-0.138889,0.243056,-0.28125,0.384615,0.384615,-0.574074,-0.19537,0.244762,0.333333,0.384615,-0.071065,-0.073565,-0.04899,0.224242,-0.006734,0.441198,0.156860,-0.319032,...,-0.203819,0.195089,0.040391,-0.181433,-0.405132,-0.428449,-0.113068,0.009834,0.149467,0.111007,0.098965,0.358279,0.268932,-0.097443,0.384615,-0.097037,-0.00283,-0.088900,-1.615385,-0.088113,0.113902,-0.318582,-0.020802,-0.010354,-0.042244,0.203571,0.279514,0.384615,0.093451,0.384615,0.384615,-0.151945,-0.167312,0.078679,-0.261950,-0.045833,-0.217857,-0.161554,0.265863,-0.197917
1004,-0.090278,0.075,-0.1875,-0.249016,-0.238095,0.225,-0.022222,0.18287,-0.168889,0.217593,0.304167,0.356481,-0.425,0.078333,-0.185185,0.043333,0.025,-0.111111,0.075,-0.291667,0.138889,0.102566,-0.138889,0.243056,-0.28125,0.054509,0.465956,-0.574074,-0.19537,0.244762,0.333333,0.026100,-0.071065,-0.073565,-0.04899,0.224242,-0.006734,0.441198,0.156860,-0.319032,...,-0.203819,0.195089,0.040391,-0.181433,-0.405132,-0.428449,-0.113068,0.009834,0.149467,0.111007,0.098965,0.358279,0.268932,-0.097443,0.212522,-0.875000,0.12500,-0.088900,-0.296135,-0.088113,0.113902,-0.318582,-0.020802,-0.010354,-0.042244,0.203571,0.279514,0.193323,0.093451,0.033527,-0.114033,-0.151945,-0.167312,0.078679,-0.261950,-0.045833,-0.217857,0.125000,0.265863,-0.197917
1005,-0.090278,0.075,-0.1875,-0.249016,-0.238095,0.225,-0.022222,0.18287,-0.168889,0.217593,0.304167,0.356481,-0.425,0.078333,-0.185185,0.043333,0.025,-0.111111,0.075,-0.291667,0.138889,0.102566,-0.138889,0.243056,-0.28125,0.054509,0.465956,-0.574074,-0.19537,0.244762,0.333333,0.026100,-0.333333,-0.073565,-0.04899,0.224242,-0.006734,0.4411

In [ ]:
final_user.head(10)

placeID,132560,132561,132564,132572,132583,132584,132594,132608,132609,132613,132626,132630,132654,132660,132663,132665,132667,132668,132706,132715,132717,132723,132732,132733,132740,132754,132755,132766,132767,132768,132773,132825,132830,132834,132845,132846,132847,132851,132854,132856,...,135044,135045,135046,135047,135048,135049,135050,135051,135052,135053,135054,135055,135057,135058,135059,135060,135062,135063,135064,135065,135066,135069,135070,135071,135072,135073,135074,135075,135076,135079,135080,135081,135082,135085,135086,135088,135104,135106,135108,135109
userID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1001,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,8.888889e-01,-1.111111e-01,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,...,-4.934325e-17,-1.111111e-01,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-1.111111e-01,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-1.111111e+00,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17,-4.934325e-17
1002,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,6.000000e-01,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,...,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,-4.000000e-01,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,-4.000000e-01,8.881784e-17,-4.000000e-01,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,8.881784e-17,-4.000000e-01,8.881784e-17,8.881784e-17,8.881784e-17,-4.000000e-01,8.881784e-17,8.881784e-17
1003,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,3.846154e-01,-3.416071e-17,-3.416071e-17,-3.416071e-17,3.846154e-01,3.846154e-01,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,3.846154e-01,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,...,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,3.846154e-01,-3.416071e-17,-3.416071e-17,-3.416071e-17,-1.615385e+00,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,3.846154e-01,-3.416071e-17,3.846154e-01,3.846154e-01,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17,-3.416071e-17
1004,0.000000e+00,0.000000e+00,0.0000

In [ ]:
# user similarity on replacing NAN by user avg
b = cosine_similarity(final_user)
np.fill_diagonal(b, 0 )
similarity_with_user = pd.DataFrame(b,index=final_user.index)
similarity_with_user.columns=final_user.index
similarity_with_user.head(10)

userID,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,1030,1031,1032,1033,1034,1035,1036,1037,1038,1039,1040,...,1099,1100,1101,1102,1103,1104,1105,1106,1107,1108,1109,1110,1111,1112,1113,1114,1115,1116,1117,1118,1119,1120,1121,1122,1123,1124,1125,1126,1127,1128,1129,1130,1131,1132,1133,1134,1135,1136,1137,1138
userID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1001,0.000000e+00,3.713378e-01,7.561126e-02,-2.936117e-33,1.089534e-02,-1.191974e-01,-1.540834e-02,6.719786e-32,-8.919915e-02,-7.194598e-34,-1.507655e-31,9.062503e-32,2.067246e-01,-1.951231e-02,2.645376e-33,-6.280743e-02,7.917379e-34,5.512655e-02,0.0,0.0,0.0,-2.227951e-01,1.380081e-33,-1.265924e-01,-3.854013e-01,0.0,-6.232980e-02,0.0,-7.677557e-03,-4.940207e-33,0.0,0.000000e+00,-1.327713e-01,1.331195e-34,2.169442e-31,3.963762e-02,-2.260939e-02,9.837747e-32,-1.507655e-31,1.380081e-33,...,2.682247e-03,0.0,-1.058735e-02,0.0,2.763659e-33,-6.342020e-02,0.0,-1.162986e-01,0.0,1.687899e-01,-2.067246e-02,-5.527894e-34,7.417313e-32,-8.744953e-33,-0.037010,-7.433262e-02,-2.075565e-02,8.538490e-02,-7.917379e-34,-5.602588e-32,9.062503e-32,3.317920e-01,-5.527894e-34,-5.689906e-02,2.629139e-33,-6.890819e-03,-1.109837e-31,5.691104e-02,0.0,0.0,-1.606627e-34,5.527894e-34,5.509409e-32,-2.109874e-01,1.095786e-31,-2.117303e-01,0.0,1.123543e-31,-9.691397e-03,1.384396e-31
1002,3.713378e-01,0.000000e+00,-3.061066e-01,-6.900656e-02,-6.454972e-02,-8.827348e-02,-3.042903e-02,-3.465131e-31,-1.045916e-01,-1.857676e-33,4.119019e-31,-2.330905e-31,2.041241e-01,1.623593e-31,-4.831028e-33,-5.168114e-02,2.536596e-33,1.088662e-01,0.0,0.0,0.0,-3.718183e-02,-3.393111e-33,-6.666667e-02,-2.752939e-01,0.0,-6.154575e-02,0.0,-1.137147e-01,6.760657e-33,0.0,0.000000e+00,2.383656e-02,6.380169e-36,-4.610714e-31,-4.696682e-02,1.461408e-31,2.777460e-01,4.119019e-31,-3.393111e-33,...,2.865088e-31,0.0,-1.411313e-01,0.0,-2.913876e-33,-1.252449e-01,0.0,1.290715e-01,0.0,6.666667e-02,2.041241e-02,2.247679e-33,-6.659188e-32,3.283603e-01,-0.014618,-5.504819e-02,5.970218e-34,9.234044e-02,-2.536596e-33,2.176561e-31,-3.070462e-31,1.297498e-01,2.247679e-33,-2.247333e-02,-6.290425e-33,-9.270068e-32,1.376205e-02,-3.746343e-02,0.0,0.0,-3.116228e-34,-2.247679e-33,-2.711357e-31,-1.000000e-01,-2.155386e-31,-1.632441e-01,0.0,2.000000e-01,4.269480e-18,3.689324e-01
1003,7.561126e-02,-3.061066e-01,0.000000e+00,2.135166e-35,1.012051e-01,-4.119069e-02,-4.781332e-32,4.416844e-32,-9.987070e-02,2.387501e-01,-7.819902e-32,3.617213e-32,4.679498e-18,-4.893120e-32,7.063170e-34,-1.852090e-02,5.238456e-34,2.557690e-32,0.0,0.0,0.0,2.293784e-32,7.830128e-34,5.972811e-02,-2.297153e-01,0.0,1.378504e-02,0.0,2.886585e-02,-2.273293e-33,0.0,7.228937e-02,2.669462e-02,5.902402e-02,6.877498e-32,-3.506549e-03,-3.620569e-32,-1.658925e-01,-7.819902e-32,7.830128e-34,...,-3.631191e-32,0.0,3.196187e-01,0.0,2.580410e-34,5.610479e-02,0.0,-2.635866e-02,0.0,1.119902e-01,2.285990e-02,-1.445278e-33,1.486284e-32,1.027112e-01,0.008185,3.082429e-02,1.414874e-34,-3.596948e-02,-5.238456e-34,-2.392011e-32,3.617213e-32,2.619282e-32,-1.445278e-33,6.065470e-01,-1.448826e-33,2.845141e-32,-4.183538e-32,-1.762126e-01,0.0,0.0,1.553385e-33,1.445278e-33,3.146097e-32,3.733007e-02,4.932489e-32,-1.019928e-01,0.0,1.754513e-01,3.643742e-02,5.902402e-02
1004,-2.936117e-33,-6.900656e-02,2.135166e-35,0.000000e+00,4.454354e-02,-3.045725e-02,1.574852e-02,2.604003e-33,1.595448e-01,0.000000e+00,9.323031e-33,-1.820073e-33,2.958040e-01,-1.857386e-34,0.000000e+00,-4.279605e-02,0.000000e+00,-4.225771e-02,0.0,0.0,0.0,6.414475e-02,0.000000e+00,1.552648e-01,-5.587377e-03,0.0,1.274118e-02,0.0,9.050162e-35,0.000000e+00,0.0,-6.681531e-02,2.467319e-02,0.000000e+00,-5.925075e-33,0.000000e+00,1.382117e-33,-1.624411e-33,9.323031e-33,0.000000e+00,...,-3.838052e-01,0.0,-3.246327e-02,0.0,0.000000e+00,3.241019e-02,0.0,-5.501248e-02,0.0,1.725164e-02,4.225771e-02,0.000000e+00,-1.683115e

In [ ]:
# user similarity on replacing NAN by item(restaurant) avg
cosine = cosine_similarity(final_restaurant)
np.fill_diagonal(cosine, 0 )
similarity_with_restaurant = pd.DataFrame(cosine,index=final_restaurant.index)
similarity_with_restaurant.columns=final_user.index
similarity_with_restaurant.head(10)

In [ ]:
#หา Top 30 user ทีเหมือนกัน
def find_n_neighbours(df,n):
    order = np.argsort(df.values, axis=1)[:, :n]
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending=False)
           .iloc[:n].index, 
          index=['top{}'.format(i) for i in range(1, n+1)]), axis=1)
    return df

In [ ]:
# top 30 neighbours for each user
sim_user_30_m = find_n_neighbours(similarity_with_user,30)
sim_user_30_m.head(10)

In [ ]:
# top 30 neighbours for each user
sim_user_30_m = find_n_neighbours(similarity_with_restaurant,30)
sim_user_30_m.head(10)

,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10,top11,top12,top13,top14,top15,top16,top17,top18,top19,top20,top21,top22,top23,top24,top25,top26,top27,top28,top29,top30
userID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1001,1092,1062,1049,1105,1135,1002,1031,1047,1051,1107,1100,1021,1079,1019,1050,1073,1128,1039,1138,1011,1043,1127,1094,1074,1013,1060,1108,1093,1120,1122
1002,1138,1062,1135,1049,1031,1047,1051,1107,1100,1021,1073,1050,1079,1043,1039,1011,1127,1094,1074,1019,1060,1038,1128,1093,1122,1110,1042,1105,1069,1070
1003,1122,1031,1047,1051,1049,1107,1100,1062,1021,1050,1019,1059,1043,1079,1127,1094,1074,1039,1138,1011,1060,1095,1135,1093,1069,1110,1042,1070,1102,1082
1004,1031,1049,1047,1062,1051,1107,1100,1021,1071,1019,1050,1079,1043,1039,1138,1011,1127,1094,1135,1074,1073,1128,1060,1093,1122,1110,1042,1069,1070,1007
1005,1094,1049,1031,1047,1062,1051,1107,1100,1021,1124,1135,1050,1073,1128,1079,1007,1043,1127,1039,1138,1011,1074,1019,1105,1109,1060,1093,1069,1110,1042
1006,1096,1031,1049,1062,1051,1107,1100,1021,1019,1050,1135,1079,1043,1127,1039,1011,1094,1138,1074,1047,1073,1057,1060,1128,1122,1093,1110,1042,1069,1070
1007,1031,1073,1049,1047,1062,1051,1107,1100,1094,1021,1128,1135,1050,1079,1043,1039,1138,1011,1127,1074,1122,1060,1069,1093,1110,1042,1070,1019,1082,1063
1008,1094,1031,1047,1049,1062,1051,1107,1100,1021,1079,1019,1050,1039,1138,1011,1073,1128,1135,1043,1127,1074,1060,1069,1093,1122,1042,1110,1007,1070,1052
1009,1019,1031,1047,1062,1051,1107,1100,1021,1050,1086,1079,1043,1127,1094,1039,1138,1011,1128,1074,1122,1049,1004,1064,1060,1073,1038,1093,1069,1110,1042


In [ ]:
#ตรวจสอบคล้ายคลึงกันในการให้คะแนนนของผู้ใช้ทั้ง 2 คน 
def get_user_similar_restaurant( user1, user2 ):
    common_restaurant = Rating_avg[Rating_avg.userID == user1].merge(
    Rating_avg[Rating_avg.userID == user2],
    on = "placeID",
    how = "inner" )
    return common_restaurant.merge( df_restaurant_cuisine, on = 'placeID' )

In [ ]:
a = get_user_similar_restaurant(1001,1002)
a = a.loc[ : , ['rating_x_x','rating_x_y','Rcuisine']]
a.head(10)

In [ ]:
#ทำนายว่าผู้ใช้จะให้คะแนนสถานที่เท่าไร
def User_item_score(user,item):
    a = sim_user_30_m[sim_user_30_m.index==user].values
    b = a.squeeze().tolist()
    c = final_restaurant.loc[:,item]
    d = c[c.index.isin(b)]
    f = d[d.notnull()]
    avg_user = Mean.loc[Mean['userID'] == user,'rating'].values[0]
    index = f.index.values.squeeze().tolist()
    corr = similarity_with_restaurant.loc[user,index]
    fin = pd.concat([f, corr], axis=1)
    fin.columns = ['adg_score','correlation']
    fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1)
    nume = fin['score'].sum()
    deno = fin['correlation'].sum()
    final_score = avg_user + (nume/deno)
    return final_score
    

In [ ]:
score = User_item_score(1001,132613)
print("score (u,i) is",score)

In [ ]:
Rating_avg = Rating_avg.astype({"placeID": str})
restaurant_user = Rating_avg.groupby(by = 'userID')['placeID'].apply(lambda x:','.join(x))
restaurant_user.head(10)

In [ ]:
#แนะนำรานอาหารให้กับผู้ใช้
def User_item_score1(user):
    restaurant_seen_by_user = check.columns[check[check.index==user].notna().any()].tolist()
    a = sim_user_30_m[sim_user_30_m.index==user].values
    b = a.squeeze().tolist()
    d = restaurant_user[restaurant_user.index.isin(b)]
    l = ','.join(d.values)
    restaurant_seen_by_similar_users = l.split(',')
    restaurant_under_consideration = list(set(restaurant_seen_by_similar_users)-set(list(map(str, restaurant_seen_by_user))))
    restaurant_under_consideration = list(map(int, restaurant_under_consideration))
    score = []
    for item in restaurant_under_consideration:
        c = final_restaurant.loc[:,item]
        d = c[c.index.isin(b)]
        f = d[d.notnull()]
        avg_user = Mean.loc[Mean['userID'] == user,'rating'].values[0]
        index = f.index.values.squeeze().tolist()
        corr = similarity_with_restaurant.loc[user,index]
        fin = pd.concat([f, corr], axis=1)
        fin.columns = ['adg_score','correlation']
        fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1)
        nume = fin['score'].sum()
        deno = fin['correlation'].sum()
        final_score = avg_user + (nume/deno)
        score.append(final_score)
    data = pd.DataFrame({'placeID':restaurant_under_consideration,'score':score})
    top_5_recommendation = data.sort_values(by='score',ascending=False).head(5)
    restaurant_Name = top_5_recommendation.merge(df_restaurant_cuisine, how='inner', on='placeID')
    restaurant_Names = restaurant_Name.title.values.tolist()
    return restaurant_Names

ข้อมูลที่ได้จะประมาณ Top 5 ร้านอาหารแนะนำ


In [ ]:
# user = int(input("Enter the user id to whom you want to recommend : "))
predicted_restaurant = User_item_score1(int(1001))
# print("score (u,i) is",predicted_restaurant)
# print(" ")
# print("The Recommendations for User Id : ")
# print("   ")
# for i in predicted_restaurant:
#     print(i)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(ratings, test_size = 0.30, random_state = 42)

print(X_train.shape)
print(X_test.shape)

Start 2

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(df_restaurant, test_size = 0.30, random_state = 42)

print(X_train.shape)
print(X_test.shape)

In [ ]:
user_data = X_train.pivot(index = 'userID', columns = 'placeID', values = 'rating').fillna(0)
user_data.head()

placeID,132560,132561,132564,132572,132583,132584,132594,132608,132609,132613,132626,132630,132654,132660,132663,132665,132667,132668,132706,132715,132717,132723,132732,132733,132740,132754,132755,132766,132767,132768,132773,132825,132830,132834,132845,132846,132847,132851,132854,132856,...,135044,135045,135046,135047,135048,135049,135050,135051,135052,135053,135054,135055,135057,135058,135059,135060,135062,135063,135064,135065,135066,135069,135070,135071,135072,135073,135074,135075,135076,135079,135080,135081,135082,135085,135086,135088,135104,135106,135108,135109
userID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
1005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
dummy_train = X_train.copy()
dummy_test = X_test.copy()

dummy_train['rating'] = dummy_train['rating'].apply(lambda x: 0 if x > 0 else 1)
dummy_test['rating'] = dummy_test['rating'].apply(lambda x: 1 if x > 0 else 0)

In [ ]:
dummy_train = dummy_train.pivot(index = 'userID', columns = 'placeID', values = 'rating').fillna(1)

# The movies not rated by user is marked as 0 for evaluation 
dummy_test = dummy_test.pivot(index ='userID', columns = 'placeID', values = 'rating').fillna(0)

In [ ]:
dummy_train.head()

In [ ]:
dummy_train['user_index'] = np.arange(0, dummy_train.shape[0],1)
dummy_train.head()

In [ ]:
dummy_train.set_index(['user_index'], inplace = True)
dummy_train.head()

placeID,132560,132561,132564,132572,132583,132584,132594,132608,132609,132613,132626,132630,132654,132660,132663,132665,132667,132668,132706,132715,132717,132723,132732,132733,132740,132754,132755,132766,132767,132768,132773,132825,132830,132834,132845,132846,132847,132851,132854,132856,...,135044,135045,135046,135047,135048,135049,135050,135051,135052,135053,135054,135055,135057,135058,135059,135060,135062,135063,135064,135065,135066,135069,135070,135071,135072,135073,135074,135075,135076,135079,135080,135081,135082,135085,135086,135088,135104,135106,135108,135109
user_index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [ ]:
user_similarity = cosine_similarity(user_data)
user_similarity[np.isnan(user_similarity)] = 0
print(user_similarity)
print(user_similarity.shape)

[[1.         0.31622777 0.22086305 ... 0.         0.01046418 0.        ]
 [0.31622777 1.         0.34921515 ... 0.23012788 0.23659807 0.22524488]
 [0.22086305 0.34921515 1.         ... 0.31213899 0.33049452 0.31233908]
 ...
 [0.         0.23012788 0.31213899 ... 1.         0.99823751 0.99930536]
 [0.01046418 0.23659807 0.33049452 ... 0.99823751 1.         0.99887741]
 [0.         0.22524488 0.31233908 ... 0.99930536 0.99887741 1.        ]]
(137, 137)


In [ ]:
user_predicted_ratings = np.dot(user_similarity, user_data)
user_predicted_ratings

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 2.52087174e+01],
       [2.23448193e-01, 4.45289953e-01, 4.44522116e-01, ...,
        2.02659560e+00, 8.86861366e-01, 2.11680318e+03],
       [3.12125974e-01, 6.22007987e-01, 6.20935426e-01, ...,
        2.75367256e+00, 1.23882169e+00, 2.93311629e+03],
       ...,
       [9.98623539e-01, 1.99006769e+00, 1.98663611e+00, ...,
        8.81016789e+00, 3.96351667e+00, 9.28715415e+03],
       [9.98195883e-01, 1.98921545e+00, 1.98578535e+00, ...,
        8.80867844e+00, 3.96181931e+00, 9.28459479e+03],
       [9.99263684e-01, 1.99134338e+00, 1.98790960e+00, ...,
        8.81581544e+00, 3.96605739e+00, 9.29215140e+03]])

In [ ]:
user_predicted_ratings.shape

(137, 131)

In [ ]:
user_final_ratings = np.multiply(user_predicted_ratings, dummy_train)
user_final_ratings.tail()

In [ ]:
user_final_ratings.iloc[100].sort_values(ascending = False)[0:5]

test


In [ ]:
test_user_features = X_test.pivot(index = 'userID', columns = 'placeID', values = 'rating').fillna(0)
test_user_similarity = cosine_similarity(test_user_features)
test_user_similarity[np.isnan(test_user_similarity)] = 0

print(test_user_similarity)
print("- "*10)
print(test_user_similarity.shape)

In [ ]:
user_predicted_ratings_test = np.dot(test_user_similarity, test_user_features)
user_predicted_ratings_test

In [ ]:
test_user_final_rating = np.multiply(user_predicted_ratings_test, dummy_test)
test_user_final_rating.head()

In [ ]:
df_restaurant['rating'].describe()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

X = test_user_final_rating.copy() 
X = X[X > 0] # only consider non-zero values as 0 means the user haven't rated the movies

scaler = MinMaxScaler(feature_range = (0.5, 5))
scaler.fit(X)
pred = scaler.transform(X)

print(pred)

In [ ]:
total_non_nan = np.count_nonzero(~np.isnan(pred))
total_non_nan

In [ ]:
test = X_test.pivot(index = 'userID', columns = 'placeID', values = 'rating')
test.head()

In [ ]:
mae = np.abs(pred - test).sum().sum()/total_non_nan
print(mae)

In [ ]:
diff_sqr_matrix = (test - pred)**2
sum_of_squares_err = diff_sqr_matrix.sum().sum() 
rmse = np.sqrt(sum_of_squares_err/total_non_nan)
print(rmse)

คั่น

In [ ]:
final_restaurant.head(10)

In [ ]:
final_restaurant.shape

In [ ]:
# user similarity on replacing NAN by item(restaurant) avg
cosine = cosine_similarity(final_restaurant)
np.fill_diagonal(cosine, 0 )
similarity_with_restaurant = pd.DataFrame(cosine,index=final_restaurant.index)
similarity_with_restaurant.columns=final_user.index
similarity_with_restaurant.head()

In [ ]:
similarity_with_restaurant.tail()

In [ ]:
similarity_with_restaurant.shape

In [ ]:
user_predicted_ratings = np.dot(similarity_with_restaurant, final_restaurant)
user_predicted_ratings

In [ ]:
user_predicted_ratings.shape

In [ ]:
dummy_train.shape

In [ ]:
dummy_test.shape

In [ ]:
user_final_ratings = np.multiply(user_predicted_ratings,dummy_train)
user_final_ratings.head(10)

end 2

In [ ]:
Rating_avg.head()

In [ ]:
pivot_data = df_restaurant.pivot(index = 'userID', columns = 'placeID', values = 'rating').fillna(0)
pivot_data.shape
pivot_data.head()

placeID,132560,132561,132564,132572,132583,132584,132594,132608,132609,132613,132626,132630,132654,132660,132663,132665,132667,132668,132706,132715,132717,132723,132732,132733,132740,132754,132755,132766,132767,132768,132773,132825,132830,132834,132845,132846,132847,132851,132854,132856,...,135044,135045,135046,135047,135048,135049,135050,135051,135052,135053,135054,135055,135057,135058,135059,135060,135062,135063,135064,135065,135066,135069,135070,135071,135072,135073,135074,135075,135076,135079,135080,135081,135082,135085,135086,135088,135104,135106,135108,135109
userID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
1005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
pivot_data['user_index'] = np.arange(0, pivot_data.shape[0],1)
pivot_data.head()

placeID,132560,132561,132564,132572,132583,132584,132594,132608,132609,132613,132626,132630,132654,132660,132663,132665,132667,132668,132706,132715,132717,132723,132732,132733,132740,132754,132755,132766,132767,132768,132773,132825,132830,132834,132845,132846,132847,132851,132854,132856,...,135045,135046,135047,135048,135049,135050,135051,135052,135053,135054,135055,135057,135058,135059,135060,135062,135063,135064,135065,135066,135069,135070,135071,135072,135073,135074,135075,135076,135079,135080,135081,135082,135085,135086,135088,135104,135106,135108,135109,user_index
userID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1
1003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
1004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,3
1005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4


In [ ]:
pivot_data.set_index(['user_index'], inplace = True)
pivot_data.head()

placeID,132560,132561,132564,132572,132583,132584,132594,132608,132609,132613,132626,132630,132654,132660,132663,132665,132667,132668,132706,132715,132717,132723,132732,132733,132740,132754,132755,132766,132767,132768,132773,132825,132830,132834,132845,132846,132847,132851,132854,132856,...,135044,135045,135046,135047,135048,135049,135050,135051,135052,135053,135054,135055,135057,135058,135059,135060,135062,135063,135064,135065,135066,135069,135070,135071,135072,135073,135074,135075,135076,135079,135080,135081,135082,135085,135086,135088,135104,135106,135108,135109
user_index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
final_user = final.apply(lambda row: row.fillna(row.mean()), axis=1)
final_user.head(10)

In [ ]:

from scipy.sparse.linalg import svds

#SVD
U,s, VT = svds(pivot_data, k = 10)

#Construct diagonal array in SVD
sigma = np.diag(s)

#Applying SVD would output 3 parameters namely
print("U = ",U) #Orthogonal matrix
print('************************************************')
print("S = ",s) #Singular values
print('************************************************')
print("VT = ", VT) #Transpose of Orthogonal matrix

In [ ]:
all_user_predicted_ratings = np.dot(np.dot(U,sigma), VT)

#Predicted ratings
pred_data = pd.DataFrame(all_user_predicted_ratings, columns = pivot_data.columns)
pred_data.head()

In [ ]:

def recommend_places(userID, pivot_data, pred_data, num_recommendations):
    user_index  = userID-1 #index starts at 0

    sorted_user_ratings = pivot_data.iloc[user_index].sort_values(ascending = False) #sort user ratings

    sorted_user_predictions = pred_data.iloc[user_index].sort_values(ascending = False)#sorted_user_predictions
    
    temp = pd.concat([sorted_user_ratings, sorted_user_predictions], axis = 1)
    temp.index.name = 'Recommended Places'
    temp.columns = ['user_ratings', 'user_predictions']
    
    temp = temp.loc[temp.user_ratings == 0]
    temp = temp.sort_values('user_predictions', ascending = False)
    print('\n Below are the recommended places for user(user_id = {}):\n'. format(userID))
    print(temp.head(num_recommendations))

In [ ]:
userID = 12
num_recommedations = 5
recommend_places(userID, pivot_data, pred_data, num_recommedations)